In [117]:
import numpy as np
import math
from scipy.spatial import cKDTree

In [71]:
def getEigenvalues(matrix):
    cov = np.cov(matrix.T)
    eigenValue = np.linalg.eigvals(cov)
    eigenValue = -np.sort(-eigenValue)
    return eigenValue/sum(eigenValue)

In [153]:
data = np.genfromtxt ("/home/augie/Ding/point cloud/sss.csv", delimiter=",", usecols=(0,1,2))

In [154]:
cdataKD = cKDTree(data, leafsize=5000)

In [155]:
features = []
start = time.time()
p1 = len(data)/100
p2 = 0
t1 = time.time()
t2 = 0
for i in range(len(data)):
    neighborIndex = cdataKD.query_ball_point(data[i],1)
    if len(neighborIndex) >= 3:
        neighbors = np.array([data[j] for j in neighborIndex])
        
        eigenvalues = getEigenvalues(neighbors)

        EV1 = eigenvalues[0]
        EV2 = eigenvalues[1]
        EV3 = eigenvalues[2]
    
        feature = [(EV1-EV2)/EV1, (EV2-EV3)/EV1, EV3/EV1]
        features.append(feature)
    else:
        features.append([0,0,0])
    
    
    if (i > p1):
        p2 += 1
        p1 += len(data)/100
        t2 = time.time()
        print(str(p2) + "%  time for this round: " + str(t2-t1))
        t1 = time.time()
        
        
end = time.time()
print("done, total time: "+  str(end-start))

1%  time for this round: 1.0421497821807861
2%  time for this round: 0.8840594291687012
3%  time for this round: 0.9173407554626465
4%  time for this round: 0.9341185092926025
5%  time for this round: 0.9676573276519775
6%  time for this round: 0.9609460830688477
7%  time for this round: 1.1293411254882812
8%  time for this round: 1.1258225440979004
9%  time for this round: 1.2146661281585693
10%  time for this round: 1.198840618133545
11%  time for this round: 1.320478916168213
12%  time for this round: 1.3469455242156982
13%  time for this round: 1.0757877826690674
14%  time for this round: 1.6704885959625244
15%  time for this round: 1.118114948272705
16%  time for this round: 1.695221185684204
17%  time for this round: 1.0019700527191162
18%  time for this round: 1.7806520462036133
19%  time for this round: 1.0591351985931396
20%  time for this round: 1.4866743087768555
21%  time for this round: 1.8816170692443848
22%  time for this round: 1.3180906772613525
23%  time for this roun

In [193]:
import csv as c
with open("sss_out_1.csv", "w") as f:
    writer = c.writer(f)
    for i in range(len(features)):
        writer.writerow(list(data[i]) + features[i])

In [64]:
class Voxel():
    #size of each voxel
    size = None
    #raw input numpy array (n * 3)
    rawData = None
    #translated numpy array (3 * n), minimal value = 0
    data = None
    #min, max and range of x, y and z
    shapes = None
    #range of x, y and z
    ranges = None
    #voxelized input data (numpy array xRange * yRange * zRange)
    voxels = None
    #
    pointsList = None
    def __init__(self, rawData, size = 1):
        self.rawData = rawData
        self.size = size
        self.getNewShapes()
        self.ranges = (self.shapes['x_range'], self.shapes['y_range'], self.shapes['z_range'])
        self.voxels = np.zeros(self.ranges)
        self.pointsList = self.makeList(self.voxels.size)
        self.loadDataToPointlist()
        #self.loadPointlistToVoxels(threshold = 0)
    def getNewShapes(self):

        data = np.copy(self.rawData.T)
        xMax, yMax, zMax = [max(data[i]) for i in range(data.shape[0])]
        xMin, yMin, zMin = [min(data[i]) for i in range(data.shape[0])]
        x_range = int((xMax - xMin) / self.size) + 1
        y_range = int((yMax - yMin) / self.size) + 1
        z_range = int((zMax - zMin) / self.size) + 1

        data[0] -= xMin
        data[1] -= yMin
        data[2] -= zMin

        shapes = {"xMax": xMax, "yMax": yMax, "zMax": zMax,
                      "xMin": xMin, "yMin": yMin, "zMin": zMin, 
                      "x_range": x_range, "y_range": y_range, "z_range": z_range}
        self.data = data
        self.shapes = shapes
    def makeList(self, length):
        return [None] * length
    def getIndexFromXYZ(self, x, y, z):
    
        y_r = self.shapes['y_range']
        z_r = self.shapes['z_range']
        return x * y_r * z_r + y * z_r + z
    
    def loadDataToPointlist(self):
        size = self.size
        voxel = self.data / size
        
        #shape[1]: number of points
        for i in range(self.data.shape[1]):
            x, y, z = int(voxel[0][i]), int(voxel[1][i]), int(voxel[2][i])
            index = self.getIndexFromXYZ(x, y, z)

            if not self.pointsList[index]:
                self.pointsList[index] = list()
            self.pointsList[index].append(i)
    def getXYZFromIndex(self, index, voxelShapes):
        y_r = voxelShapes['y_range']
        z_r = voxelShapes['z_range']
        x, tmp = divmod(index, y_r * z_r)
        y, z = divmod(tmp, z_r)

        return x, y, z
    
    def loadPointlistToVoxels(self, threshold = 3):
        for i in range(len(self.pointsList)):
            if self.pointsList[i]:
                if (len(self.pointsList[i]) > threshold):
                    x, y, z = self.getXYZFromIndex(i, self.shapes)
                    self.voxels[x][y][z] = 1
    def getPointsFromIJK(self, i, j, k):
        index = self.getIndexFromXYZ(i, j, k)
        pointsIndex = self.pointsList[index]
        tmp = None
        if (pointsIndex):
            tmp = np.zeros((len(pointsIndex), 3))
            for q in range(len(pointsIndex)):
                tmp[q] = self.rawData[pointsIndex[q]]
        return tmp
    
    def getPointsListFromIJK(self, i, j, k):
        index = self.getIndexFromXYZ(i, j, k)
        pointsIndex = self.pointsList[index]
        tmp = []
        if (pointsIndex):
            for q in range(len(pointsIndex)):
                tmp.append(list(self.rawData[pointsIndex[q]]))
        return tmp
    
    
    def getIJKFromPoint(self, x, y, z):
        xMin = self.shapes["xMin"]
        yMin = self.shapes["yMin"]
        zMin = self.shapes["zMin"]
        return (int((x-xMin)/self.size),int((y-yMin)/self.size),int((z-zMin)/self.size))
    
    
    def checkDistance(self, x, y, z, cx, cy, cz, r):
        if math.pow(x-cx, 2) + math.pow(y-cy, 2) + math.pow(z-cz, 2) < math.pow(r, 2):
            return True
        else:
            return False
        
    
    def getSurroundingPoints(self, x, y, z, radius = 3):
        global total
        t1=0
        t2=0
        
        center = self.getIJKFromPoint(x, y, z)
        voxelNumber = int(radius/self.size) + 1
        xMin = max(0, center[0] - voxelNumber)
        xMax = min(self.ranges[0], center[0] + voxelNumber)
        yMin = max(0, center[1] - voxelNumber)
        yMax = min(self.ranges[1], center[1] + voxelNumber)
        zMin = max(0, center[2] - voxelNumber)
        zMax = min(self.ranges[2], center[2] + voxelNumber)
        
        c = int((radius/self.size)/1.4143) - 1
        xMustMin = center[0] - c
        xMustMax = center[0] + c
        yMustMin = center[1] - c
        yMustMax = center[1] + c
        zMustMin = center[2] - c
        zMustMax = center[2] + c
        
        
        surroundingPoints = []
        #no need to -1 for max part
        for i in range(xMin, xMax):
            
            if xMustMin <= i <= xMustMax:
                for j in range(yMin, yMax):
                    if yMustMin <= j <= yMustMax:
                        for k in range(zMin, zMax):
                            if zMustMin <= k <= zMustMax:
                                
                                surroundingPoints += self.getPointsListFromIJK(i, j, k)
                                
                            else:
                                
                                potential = self.getPointsListFromIJK(i, j, k)
                                for xx, yy, zz in potential:
                                    if self.checkDistance(xx, yy, zz, x, y, z, radius):
                                        surroundingPoints.append([xx, yy, zz])
                                
                                
                    else:
                        
                        for k in range(zMin, zMax):
                            potential = self.getPointsListFromIJK(i, j, k)
                            for xx, yy, zz in potential:
                                if self.checkDistance(xx, yy, zz, x, y, z, radius):
                                    surroundingPoints.append([xx, yy, zz])
                        
            else:
                t1 = time.time()
                for j in range(yMin, yMax):
                    for k in range(zMin, zMax):
                        potential = self.getPointsListFromIJK(i, j, k)
                        for xx, yy, zz in potential:
                            if self.checkDistance(xx, yy, zz, x, y, z, radius):
                                surroundingPoints.append([xx, yy, zz])
                t2= time.time()
            total += (t2-t1)
        return surroundingPoints